**Task 1 and 2**

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [29]:
def relationship(y):
    return y**3 + y**2 - y - 1

#neural network with one linear layer
class Net_model(nn.Module):
    def __init__(self):
        super(Net_model, self).__init__()
        self.fc = nn.Linear(1, 1)

    def forward(self, i):
        return self.fc(i)
#instance
model = Net_model()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    z = torch.randn(1, requires_grad=True)
    y_pred_val = model(z)

    y_true_val = relationship(z)
    loss = criterion(y_pred_val, y_true_val)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Saving gradients and weights
gradients = z.grad.item()
weights = model.fc.weight.data.item()

#at 100 value
at_val_100 = torch.tensor([[100.]], requires_grad=True)
y_pred_at_100 = model(at_val_100)
prediction_at_value_100 = y_pred_at_100.item()

# Calculate the error
true_value_at_100 = relationship(at_val_100).item()
error_at_val_100 = true_value_at_100 - prediction_at_value_100

print(f"gradients: {gradients}", f"\nweights: {weights}", f"\nAt value 100: prediction: {prediction_at_value_100}")
print(f"At value 100: True value: {true_value_at_100}")
print(f"At value 100: Error: {error_at_val_100}")


gradients: -2.0157108306884766 
weights: -0.7207477688789368 
At value 100: prediction: -71.16104888916016
At value 100: True value: 1009899.0
At value 100: Error: 1009970.1610488892


**Task 3**

In [46]:
#Modified the Net_model class for activations
class Net_model(nn.Module):
    def __init__(self, activation):
        super(Net_model, self).__init__()
        self.fc = nn.Linear(1, 1)
        self.activation = activation

    def forward(self, x):
        x = self.fc(x)
        return self.activation(x)


#different activation functions
activations = [nn.ReLU(), nn.Tanh(), nn.Sigmoid()]

for activation in activations:
    model = Net_model(activation)
    print(f"\nTraining with {activation.__class__.__name__} activation:")

    #different loss functions
    losses = [nn.MSELoss(), nn.L1Loss()]

    for loss_func in losses:

        optimzr = optim.SGD(model.parameters(), lr=0.001)
        print(f"\nLoss function: {loss_func.__class__.__name__}")

        #training starts here
        epochs = 1000
        for epoch in range(epochs):
            x = torch.randn(1, requires_grad=True)
            y_pred_val = model(x)
            # calling relationship function
            y_true_val = relationship(x)
            loss = loss_func(y_pred_val, y_true_val)
            optimzr.zero_grad()
            loss.backward()
            optimzr.step()

        # predicting value at 100
        x_100 = torch.tensor([[100.]], requires_grad=True)
        y_pred_100 = model(x_100)
        prediction_at_val_100 = y_pred_100.item()

        #error calculation
        true_value_at_100 = relationship(x_100).item()
        error_at_100 = true_value_at_100 - prediction_at_val_100

        print(f"Prediction at 100: {prediction_at_val_100}")
        print(f"True value at 100: {true_value_at_100}")
        print(f"Error at 100: {error_at_100}")


Training with ReLU activation:

Loss function: MSELoss
Prediction at 100: 0.3099159300327301
True value at 100: 1009899.0
Error at 100: 1009898.69008407

Loss function: L1Loss
Prediction at 100: 0.3099159300327301
True value at 100: 1009899.0
Error at 100: 1009898.69008407

Training with Tanh activation:

Loss function: MSELoss
Prediction at 100: 1.0
True value at 100: 1009899.0
Error at 100: 1009898.0

Loss function: L1Loss
Prediction at 100: 1.0
True value at 100: 1009899.0
Error at 100: 1009898.0

Training with Sigmoid activation:

Loss function: MSELoss
Prediction at 100: 0.5692905783653259
True value at 100: 1009899.0
Error at 100: 1009898.4307094216

Loss function: L1Loss
Prediction at 100: 0.9998823404312134
True value at 100: 1009899.0
Error at 100: 1009898.0001176596


**Task 4**

In [31]:
#transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

#loading MNIST dataset
training_set = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
training_loader = torch.utils.data.DataLoader(training_set, batch_size=5, shuffle=True)

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.res = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(32 * 14 * 14, 10)

    def forward(self, a):
        a = self.res(a)
        a = a.view(a.size(0), -1)
        a = self.fc(a)
        return a

#model instance
CNN_model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(CNN_model.parameters(), lr=0.0001)
# training
for epoch in range(3):
    epoch_loss = 0.0
    for inputs, labels in training_loader:
        #making gradients zero
        optimizer.zero_grad()
        outputs = CNN_model(inputs)
        loss = criterion(outputs, labels)
        #backward pass
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print("Epoch {}, Loss: {:.4f}".format(epoch + 1, epoch_loss / len(training_loader)))
print('Training completed...')

Epoch 1, Loss: 0.2470
Epoch 2, Loss: 0.0993
Epoch 3, Loss: 0.0742
Training completed...
